# Azure AI Agents na may Suporta sa Model Context Protocol (MCP) - Python

Ang notebook na ito ay nagpapakita kung paano gamitin ang Azure AI Agents na may mga tool ng Model Context Protocol (MCP) sa Python. Ipinapakita nito kung paano lumikha ng isang matalinong ahente na maaaring gumamit ng mga panlabas na MCP server (tulad ng Microsoft Learn) para sa mas pinahusay na kakayahan gamit ang keyless authentication.


## I-install ang Mga Kinakailangang Python Package

Una, kailangan nating i-install ang mga kinakailangang Python package:
- **azure-ai-projects**: Core Azure AI Projects SDK
- **azure-ai-agents**: Azure AI Agents SDK para sa paglikha at pamamahala ng mga agent
- **azure-identity**: Nagbibigay ng keyless authentication gamit ang DefaultAzureCredential
- **mcp**: Implementasyon ng Model Context Protocol para sa Python


## Mga Benepisyo ng Keyless Authentication

Ipinapakita ng notebook na ito ang **keyless authentication** na may maraming benepisyo:
- ✅ **Walang API keys na kailangang pamahalaan** - Gumagamit ng Azure identity-based authentication
- ✅ **Mas pinahusay na seguridad** - Walang mga lihim na nakaimbak sa code o configuration files
- ✅ **Awtomatikong pag-ikot ng kredensyal** - Ang Azure ang namamahala sa lifecycle ng mga kredensyal
- ✅ **Role-based access control** - Gumagamit ng Azure RBAC para sa mas detalyadong pahintulot
- ✅ **Suporta sa maraming environment** - Gumagana nang maayos sa development at production

Ang `DefaultAzureCredential` ay awtomatikong pumipili ng pinakamahusay na mapagkukunan ng kredensyal:
1. **Managed Identity** (kapag tumatakbo sa Azure)
2. **Azure CLI** credentials (sa panahon ng lokal na development)
3. **Visual Studio** credentials
4. **Environment variables** (kung naka-configure)
5. **Interactive browser** authentication (bilang fallback)


## Pag-setup ng Keyless Authentication

**Mga kinakailangan para sa keyless authentication:**

### Para sa Lokal na Pag-develop:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Para sa Azure Environments:
- I-enable ang **System-assigned Managed Identity** sa iyong Azure resource
- Mag-assign ng tamang **RBAC roles** sa managed identity:
  - `Cognitive Services OpenAI User` para sa access sa Azure OpenAI
  - `AI Developer` para sa access sa Azure AI Projects

### Mga Environment Variable (Opsyonal):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Hindi na kailangan ng API keys o connection strings!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## I-import ang Kinakailangang Mga Aklatan

I-import ang mga kinakailangang Python modules:  
- **os, time**: Mga karaniwang Python library para sa mga environment variable at pagkaantala  
- **AIProjectClient**: Pangunahing kliyente para sa Azure AI Projects  
- **DefaultAzureCredential**: Authentication na walang susi para sa mga serbisyo ng Azure  
- **MCP-related classes**: Para sa paglikha at pamamahala ng mga MCP tools at paghawak ng mga pag-apruba  


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## I-configure ang Mga Setting ng MCP Server

I-set up ang configuration ng MCP server gamit ang mga environment variable na may fallback defaults:
- **MCP_SERVER_URL**: Ang URL ng MCP server (default ay Microsoft Learn API)
- **MCP_SERVER_LABEL**: Isang label para makilala ang MCP server (default ay "mslearn")

Ang pamamaraang ito ay nagbibigay-daan para sa flexible na configuration sa iba't ibang mga environment.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Gumawa ng Azure AI Project Client (Keyless Authentication)

I-initialize ang Azure AI Project client gamit ang **keyless authentication**:
- **endpoint**: Ang Azure AI Foundry project endpoint URL
- **credential**: Gumagamit ng `DefaultAzureCredential()` para sa ligtas at keyless na authentication
- **Walang kinakailangang API keys**: Awtomatikong hinahanap at ginagamit ang pinakamahusay na available na credential

**Authentication Flow:**
1. Sinusuri ang Managed Identity (sa mga Azure environment)
2. Gumagamit ng Azure CLI credentials (para sa lokal na development) bilang fallback
3. Gumagamit ng iba pang available na pinagmumulan ng credential kung kinakailangan

Ang pamamaraang ito ay nag-aalis ng pangangailangan na mag-manage ng API keys o connection strings sa iyong code.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Gumawa ng MCP Tool Definition

Gumawa ng MCP tool na kumokonekta sa Microsoft Learn MCP server:
- **server_label**: Tagapagkilala para sa MCP server
- **server_url**: URL endpoint ng MCP server
- **allowed_tools**: Opsyonal na listahan para limitahan kung aling mga tool ang maaaring gamitin (ang walang laman na listahan ay nagpapahintulot sa lahat ng tool)

Ang tool na ito ay magbibigay-daan sa ahente na ma-access ang dokumentasyon at mga mapagkukunan ng Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Gumawa ng Ahente at Magpatakbo ng Usapan (Keyless Workflow)

Ang detalyadong seksyong ito ay nagpapakita ng kumpletong **keyless agent workflow**:

1. **Gumawa ng AI Ahente**: Mag-set up ng ahente gamit ang GPT-4.1 nano model at MCP tools
2. **Gumawa ng Thread**: Magtatag ng thread para sa komunikasyon
3. **Magpadala ng Mensahe**: Magtanong sa ahente tungkol sa pagkakaiba ng Azure OpenAI at OpenAI
4. **Pamahalaan ang Tool Approvals**: Awtomatikong aprubahan ang mga tawag sa MCP tools kapag kinakailangan
5. **Subaybayan ang Pagpapatakbo**: Bantayan ang progreso ng ahente at pamahalaan ang anumang kinakailangang aksyon
6. **Ipakita ang Resulta**: Ipakita ang detalye ng usapan at paggamit ng tools

**Mga Tampok ng Keyless:**
- ✅ **Walang hardcoded na mga lihim** - Lahat ng authentication ay pinangangasiwaan ng Azure identity
- ✅ **Ligtas sa default** - Gumagamit ng role-based access control
- ✅ **Pinadaling deployment** - Walang kinakailangang pamamahala ng kredensyal
- ✅ **Audit-friendly** - Lahat ng access ay sinusubaybayan sa pamamagitan ng Azure identity

Gagamitin ng ahente ang MCP tools upang ma-access ang Microsoft Learn resources nang may buong seguridad at walang pamamahala ng API key.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
